# Preprocesamiento
---

En este _notebook_ mostraremos un ejemplo práctico de preprocesamiento de textos usando un _corpus_ en español de _Kaggle_ llamado [Spanish tweets suggesting depression](https://www.kaggle.com/datasets/francescoronzano/spanish-tweets-suggesting-depression).

<img src="https://drive.google.com/uc?export=view&id=1Vh514lihPERVhRK4pxuwu157IGSvFM7k" width="60%">

## **1. Carga de Datos**

En este caso disponemos de un conjunto de datos tabular (formato `csv`) que contiene 4 columnas:

- `id`: identificador único de cada tweet.
- `user_id`: identificador único de un usuario.
- `text`: texto del tweet.
- `date`: fecha de creación del tweet.

<img src="https://drive.google.com/uc?export=view&id=1mtV9eevIWZcJHq9aWRBHkn5kIr8pHBvf" width="60%">

Vamos a cargar el conjunto de datos con `pandas`. Primero importamos la librería:

In [ ]:
import pandas as pd
from IPython.display import display

Ahora, cargamos el conjunto de datos en la variable `df`:

In [ ]:
df = pd.read_csv(
        "https://raw.githubusercontent.com/mindlab-unal/mlds4-datasets/main/u2/tweets_spa.csv",
        index_col=0
    )
display(df.head())

,id,user_id,text,date
0,1,1,Deberían eliminar a las malas personas y a los...,Sat Oct 28 16:07:06 +0000 2017
1,2,1,Ya deja de intentar contarle tus problemas a a...,Thu Nov 02 06:25:04 +0000 2017
2,3,1,La tristeza es lo más fácil de ocultar de todo...,Thu Dec 14 02:06:52 +0000 2017
3,4,1,De las peores cosas de la depresión es que no ...,Sun Jul 16 19:30:11 +0000 2017
4,5,1,La soledad es lo único constante en mi vida. C...,Mon Oct 23 02:52:07 +0000 2017


Validemos los tipos de las columnas que cargamos:

In [ ]:
display(df.dtypes)

,0
id,int64
user_id,int64
text,object
date,object


## **2. Estructura Típica de Preprocesamiento**
---

En este caso veremos algunas de las operaciones más comunes en distintos preprocesamientos de información textual. Es necesario resaltar que no en todos los casos necesitará aplicar todas las técnicas e incluso en alguna aplicación específica va a necesitar estrategias más especializadas, no obstante, lo que presentamos en este caso aplica de forma general a la gran mayoría de aplicaciones de Procesamiento de Lenguaje Natural.

Veamos un diagrama de los **componentes de preprocesamiento** que veremos en este _notebook_:

<img src="https://drive.google.com/uc?export=view&id=1LZ21z7HWloUlrZPxQPgG2656VPXgglUl" width="100%">

Para estos ejemplos diseñaremos el preprocesamiento para un único documento y, posteriormente, aplicaremos todo el **preprocesamiento** al _corpus_ al finalizar el _notebook_:

In [ ]:
text = df['text'].iloc[2]
print(text)

La tristeza es lo más fácil de ocultar de todos los sentimientos que trae consigo la depresión

#LaDepresiónEsMulticolor


## **3. Tokenizado del Texto**
---

El proceso de tokenizado consiste en separar el texto en unidades lógicas (caracteres, palabras, oraciones).

Para esto usaremos `spacy`, comenzamos importándolo:

In [ ]:
import spacy

Descargamos el _Pipeline_ para el español:

In [ ]:
spacy.cli.download("es_core_news_sm")

✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


Creamos el _Pipeline_:

In [ ]:
nlp = spacy.load("es_core_news_sm")
print(nlp)

Vamos a crear un documento de `spacy` a partir del texto:

In [ ]:
doc = nlp(text)
print(doc)

La tristeza es lo más fácil de ocultar de todos los sentimientos que trae consigo la depresión

#LaDepresiónEsMulticolor


Podemos extraer los tokens a nivel de palabras del documento:

In [ ]:
tokens = [token for token in doc]
print(tokens)

[La, tristeza, es, lo, más, fácil, de, ocultar, de, todos, los, sentimientos, que, trae, consigo, la, depresión, 

, #, LaDepresiónEsMulticolor]


Veamos el tipo de un elemento de `tokens` para validar que sea un `Token` de `spacy`:

In [ ]:
print(type(tokens[0]))

<class 'spacy.tokens.token.Token'>


## **4. Filtrado de Palabras**
---

Una práctica común en el preprocesamiento de textos es el filtrado de _tokens_ según distintas condiciones.

<img src="https://drive.google.com/uc?export=view&id=1rgVQ933f6qXPslXbRLYE57MwvVl2b5hl" width="100%">

Por ejemplo, podemos eliminar todas las palabras que sean _stopwords_, para ello definimos una condición:

In [ ]:
condition = lambda token: not token.is_stop

Filtramos los tokens:

In [ ]:
filtered_tokens = list(filter(condition, tokens))
print(filtered_tokens)

[tristeza, fácil, ocultar, sentimientos, trae, depresión, 

, #, LaDepresiónEsMulticolor]


Como se puede observar, eliminamos _tokens_ como `"a"`, `"las"`, `"y"`, `"los"`, entre otras que son muy comunes (y poco informativas) en Español.

También es posible filtrar palabras por longitud, por ejemplo, en español la palabra más larga tiene 23 letras (electroencefalografista), por ello, podemos definir una condición para filtrar palabras que tengan una longitud en un rango dado:

In [ ]:
condition = lambda token: len(token) > 0 and len(token) < 24

Filtramos los tokens:

In [ ]:
filtered_tokens2 = list(filter(condition, filtered_tokens))
print(filtered_tokens2)

[tristeza, fácil, ocultar, sentimientos, trae, depresión, 

, #, LaDepresiónEsMulticolor]


## **5. Lematización**
---

En algunas aplicaciones no es de importancia la conjugación de las palabras (tiempo, plurales, géneros, ...), por lo cual, se suele transformar el texto a sus versiones lematizadas.

Vamos a aplicar este enfoque sobre cada uno de los _tokens_ filtrados:

In [ ]:
lemmas = [token.lemma_ for token in filtered_tokens2]
print(lemmas)

['tristeza', 'fácil', 'ocultar', 'sentimiento', 'traer', 'depresión', '\n\n', '#', 'LaDepresiónEsMulticolor']


Como puede ver, palabras como `"trae"` se convierten en su infinitivo `"traer"`.

Finalmente, unimos todos los _tokens_ en un único _string_:

In [ ]:
lemma_text = " ".join(lemmas)
print(lemma_text)

tristeza fácil ocultar sentimiento traer depresión 

 # LaDepresiónEsMulticolor


## **6. Normalización de Caracteres**
---

En muchos idiomas tenemos caracteres modificadores de vocales o letras. Por ejemplo, en español tenemos las tildes y la letra "ñ"; o en el caso del portugués tenemos tres tipos de acentos.

Este tipo de variaciones pueden ser eliminadas en un proceso de normalización de los textos con distintos tipos de codificaciones. Para el español es muy común usar la librería `unidecode`, vamos a instalarla:

In [ ]:
!pip install unidecode

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 8.6 MB/s eta 0:00:00


Ahora podemos importarla:

In [ ]:
from unidecode import unidecode

Veamos cómo queda el texto luego de normalizarlo:

In [ ]:
norm_text = unidecode(lemma_text)
print(norm_text)

tristeza facil ocultar sentimiento traer depresion 

 # LaDepresionEsMulticolor


Como puede ver, ya no hay tildes en el texto.

## **7. Modificación de la Grafía**
---

Normalmente, los textos se suelen procesar en minúsculas. Con esto eliminamos modificadores relacionados con el inicio de un texto, palabras capitalizadas luego de signos de puntuación, entre otras cosas.

En este caso, convertimos el texto a minúsculas:

In [ ]:
lower_text = norm_text.lower()
print(lower_text)

tristeza facil ocultar sentimiento traer depresion 

 # ladepresionesmulticolor


## **8. Limpieza con Regex**
---

Comúnmente se suelen aplicar expresiones regulares para eliminar caracteres o secuencias de caracteres no deseadas. En el ejemplo que estamos desarrollando, podemos aplicar una expresión regular para eliminar todos los caracteres que no sean espacios ni letras minúsculas.

Primero importamos la librería para expresiones regulares:

In [ ]:
import re

Ahora, definimos una expresión regular que cumpla con los criterios mencionados:

In [ ]:
pat = re.compile(r"[^a-z ]")

Reemplazamos las coincidencias:

In [ ]:
clean_text = re.sub(pat, "", lower_text)
print(clean_text)

tristeza facil ocultar sentimiento traer depresion   ladepresionesmulticolor


Como se puede ver, ahora tenemos espacios repetidos (ya que algunos _tokens_ se eliminaron), podemos usar una segunda expresión regular para eliminar espacios repetidos:

In [ ]:
spaces = re.compile(r"\s{2,}")

Reemplazamos espacios repetidos por un único espacio:

In [ ]:
spaces_text = re.sub(spaces, " ", clean_text)
print(spaces_text)

tristeza facil ocultar sentimiento traer depresion ladepresionesmulticolor


## **9. Preprocesamiento Completo**
---

Como pudimos ver en el paso anterior, tenemos un texto más limpio que puede llegar a ser más fácil de analizar de manera automática.

A continuación veremos la aplicación del preprocesamiento sobre el _corpus_ completo. Para ello, definimos la función `preprocess` la cual toma como entrada un documento cualquiera y retorna un documento preprocesado:

In [ ]:
pat = re.compile(r"[^a-z ]")
spaces = re.compile(r"\s{2,}")

def preprocess(text, min_len=1, max_len=23):
    # Creamos documento de spacy
    doc = nlp(text)
    # Eliminamos stopwords
    filtered_tokens = filter(
            lambda token: not token.is_stop,
            doc
            )
    # Filtramos palabras por longitud
    filtered_tokens2 = filter(
            lambda token: len(token) >= min_len and len(token) <= max_len,
            filtered_tokens
        )
    # Obtenemos los lemmas de cada token
    lemmas = map(
            lambda token: token.lemma_,
            filtered_tokens2
            )
    lemma_text = " ".join(lemmas)
    # Normalizamos el texto
    norm_text = unidecode(lemma_text)
    # Quitamos grafía
    lower_text = norm_text.lower()
    # Eliminamos caracteres especiales
    clean_text = re.sub(pat, "", lower_text)
    # Eliminamos espacios duplicados
    spaces_text = re.sub(spaces, " ", clean_text)
    return spaces_text.strip()

Veamos un ejemplo de la función:

In [ ]:
prep_text = preprocess(df['text'].iloc[0])
print(prep_text)

deber eliminar mala persona odiar vivir mundo


Podemos usar el método `apply` de los dataframes de `pandas` para aplicar el preprocesamiento sobre todo el _corpus_:

In [ ]:
prep_corpus = df['text'].apply(preprocess).tolist()
print(prep_corpus[:10])

['deber eliminar mala persona odiar vivir mundo', 'dejar intentar contar el problema alguien entender importa', 'tristeza facil ocultar sentimiento traer depresion ladepresionesmulticolor', 'peor cosa depresion dejar gana vivir matarte agonizar', 'soledad unico constante vida conocerar amor amistad felicidad vivir cansar', 'querer desaparecer tiempo quisiser alguien dar infierno vivir', 'madre enferma flojera narcolepsia consumiendo depresion', 'vida sentido gana ir yo mundo fiesta externallink', 'cansar gordo cansar fracaso cansar querer irmeeeeeeeee', 'calculo deber muerto']


Como se puede ver, obtuvimos un _corpus_ más estandarizado y compacto con un flujo típico de preprocesamiento.